## Diffusion 3D image 2 image

This example shows how you can train 3D image2image with this sub-repository

Go to http://brain-development.org/ixi-dataset/ and download T1 images (all images) and T2 images (all images) and put them in the folder /data/T1wand data/T2


In [ ]:
from pathlib import Path

assert Path(
    "data/T1"
).exists(), "Go to http://brain-development.org/ixi-dataset/ and download T1 images (all images) and T2 images (all images) and put them in the folder /data/T1 and data/T2"
assert Path(
    "data/T2"
).exists(), "Go to http://brain-development.org/ixi-dataset/ and download T1 images (all images) and T2 images (all images) and put them in the folder /data/T1 and data/T2"
assert Path(
    "data/T1/IXI012-HH-1211-T1.nii.gz"
).exists(), "Go to http://brain-development.org/ixi-dataset/ and download T1 images (all images) and T2 images (all images) and put them in the folder /data/T1 and data/T2"
assert Path(
    "data/T2/IXI012-HH-1211-T2.nii.gz"
).exists(), "Go to http://brain-development.org/ixi-dataset/ and download T1 images (all images) and T2 images (all images) and put them in the folder /data/T1 and data/T2"

Make a csv or xlsx with the data pairs.

In one row we have matching images


Head: idx | Phase | Path | [NAME]*

In [ ]:
d = {"Phase": [], "Path": [], "T1w": [], "T2w": []}
import random
import nibabel as nib
import pandas as pd


def rest_path(x: Path | None):
    if x is None:
        return None
    par = x.parent.name
    return par + "/" + x.name


random.seed(42)
for t2w_path in Path("data/T2").iterdir():
    split = random.random()
    if split < 0.8:
        phase = "train"
    elif split < 0.85:
        phase = "val"
    else:
        phase = "test"
    if Path(str(t2w_path).replace("T2", "T1")).exists():
        t1w_path = Path(str(t2w_path).replace("T2", "T1_resampled"))

        d["Phase"].append(phase)
        d["Path"].append(t2w_path.parent.parent.absolute())
        d["T1w"].append(rest_path(t1w_path))
        d["T2w"].append(rest_path(t2w_path))
df = pd.DataFrame(data=d)
print(df.head())
xls = Path("data", "train_paired.xlsx")
df.to_excel(xls)

This dataset is registered but not yet resampled to the same pixel space. We fix this by using resample_from_to.
For brain translation we usually use skull-striping, to remove the brain. We skip this for simplicity.

Warning this will will take a while!



In [ ]:
import nibabel.processing as nip


for t2w_path in Path("data/T2").iterdir():
    t1w_path = Path(str(t2w_path).replace("T2", "T1"))
    t1w_path_out = Path(str(t2w_path).replace("T2", "T1_resampled"))
    t1w_path_out.parent.mkdir(exist_ok=True)
    if not t1w_path.exists():
        continue
    if t1w_path_out.exists():
        print("skip:", t1w_path, "    ", end="\r")
        continue
    t2w_nib = nib.load(t2w_path)
    t2w_arr = t2w_nib.get_fdata()
    t1w_nib: nib.nifti1.Nifti1Image = nib.load(t1w_path)
    # t1w_nib.header.get_zooms() == t2w_nib.header.get_zooms():
    t1w_nib = nip.resample_from_to(t1w_nib, t2w_nib, 0, cval=0, mode="constant")
    nib.save(t1w_nib, t1w_path_out)
    print("save:", t1w_path, "    ", end="\r")

Make a new config file

In [ ]:
a = f"""batch_size: 1
experiment_name:'Diffusion_3D_Brain_T2w_to_T1w' 
lr:0.0002 
batch_size:1 
batch_size_val:1 
num_epochs:2000 
num_cpu:16 
target_patch_shape:[96, 96, 96] #24 GB
flip:True 
gpus:[0] 
new:False 
prevent_nan:False 
volumes:True 
dim_multiples:'1, 2, 4, 8' 
channels:64 
cpu:False 
start_epoch:0 
log_dir:'logs_diffusion3D' 
model_name:'unet' 
L2_loss:False 
linear:False 
learned_variance:False #Do not use, because it it not implemented in DDIM (only in DDPM)
timesteps:1000 
image_mode:True # False: predict noise; True: predict x_0 (initial image)
conditional_dimensions:1 

image_dropout: 0.0 # Must be set 0.0 > for classifier free guidance
dataset: {str(xls)}
dataset_val: {str(xls)}

output_rows : T1w
input_rows : T2w
conditional_label_size: 0"""
f = open("configs/diffusion3D_brain.conf", "a")
f.write(a)
f.close()
print(a)

You can now start the training with 
```
python train3D.py --config configs/diffusion3D_brain.conf
```

We recommend this to start in a terminal.

If you ssh to an linux sever for your Deep Learning, consider "tmux", so you can close your terminal without stopping your training.

In [ ]:
if input("We recommend this to start in a terminal. Start anyway! (yes/no)") == "yes":
    !python train3D.py --config configs/diffusion3D_brain.conf
else:
    print('skip')

## Tensorboard


We use Tensorboard. Activate it with in a terminal:
```
cd /path/to/this/folder
tensorboard --logdir logs_diffusion3D --port 6008 --samples_per_plugin=images=100
```
cd to the git path. The **logs_diffusion3D** will be created when you run the diffusion.
You can see training and images got to "http://localhost:6008" in your web browser. The loss of diffusion converges only in the beginning of the training, the model will still improve.


In [ ]:
!tensorboard --logdir logs_diffusion3D --port 6008 --samples_per_plugin=images=100

# Inference

In [1]:
from train3D import Diffusion3D
import torch
from torch import Tensor
from utils.nii import NII
import glob
import os

files = glob.glob(
    "logs_diffusion3D/Diffusion_3D_Diffusion_3D_Brain_T2w_to_T1w/version_*/checkpoints/epoch=*-step=*-train_All=*_latest.ckpt"
)
files.sort(key=os.path.getmtime)
model_path = files[0]
print(model_path)
diffusion = Diffusion3D.load_from_checkpoint(model_path)

logs_diffusion3D/Diffusion_3D_Diffusion_3D_Brain_T2w_to_T1w/version_0/checkpoints/epoch=147-step=67488-train_All=0.00000000_latest.ckpt


In [10]:
from utils.preprocessing import pad_size3D, run_model, revert_iso_to_original


def prepare_nii(mri_path):
    nii_iso = NII.load(mri_path, False, 0)
    # nii_iso = nii.rescale((1, 1, 1), verbose=True).reorient(("R", "I", "P"))
    nii_iso /= nii_iso.max() * 1.1
    nii_iso.clamp_(min=0)
    nii_iso = nii_iso * 2 - 1
    arr = Tensor(nii_iso.get_array().astype(float))
    arr, padding = pad_size3D(arr)
    return arr, padding, nii_iso

In [11]:
def translate(mri_path, save_path=None, use_cpu=False, steps=25, eta=1):
    arr, padding, nii_iso = prepare_nii(mri_path)
    if torch.cuda.is_available() and not use_cpu:
        # If you run out of memory use max_shape= (arr.shape[-1]*arr.shape[-2]*arr.shape[-3])//2
        print(arr.max(), arr.min())
        out_arr = run_model(diffusion, conditional=arr, gpu=True, eta=eta, w=0, steps=steps)
    else:
        print("!!! Fall back to less steps on CPU instead of 25 GPU. !!!")
        out_arr = run_model(diffusion, conditional=arr, gpu=False, eta=eta, w=0, steps=steps // 10)
    out_arr += 1000
    out_arr /= 20
    other_nii_iso = nii_iso.set_array(out_arr.numpy())
    other_nii = revert_iso_to_original(other_nii_iso, None, padding)
    if save_path is not None:
        other_nii.save(save_path)
    return other_nii

In [13]:
mri_path = "data/T2/IXI054-Guys-0707-T2.nii.gz"
out_path = "data/IXI054-Guys-0707-T1_desc-translated.nii.gz"
translate(mri_path, save_path=out_path, steps=25)  # use_cpu=True

'set_array' with different dtype: from int16 to float64
tensor(1.) tensor(0.)
Run Diffusion; steps = 25, eta = 1.0, w = 0


sampling ddim eta=1.0: 100%|██████████| 25/25 [01:27<00:00,  3.48s/it]


'set_array' with different dtype: from float64 to float32
Save data/IXI054-Guys-0707-T1_desc-translated.nii.gz as float32
